In [1]:
from dotenv import load_dotenv
import os
import requests
import base64
from collections import defaultdict
from datetime import datetime


#This is a function to setup a connection to the Intervals.icu API and retrieve activities.
def get_Activities():
    url = "https://intervals.icu/api/v1/athlete/0/activities?oldest=2020-12-01&newest=2025-12-31"
    
    load_dotenv('APIKEY.env')
    # Basic auth credentials
    username = os.getenv("USERNAME")
    password = os.getenv("PASSWORD")
    
    if not username or not password:
        print("Failed to load credentials. Check if APIKEY.env exists and contains USERNAME and PASSWORD")
        return None

    
    
    # Create basic auth string and encode it
    credentials = base64.b64encode(f"{username}:{password}".encode()).decode()
    
    headers = {
        "Authorization": f"Basic {credentials}",
        "Accept": "application/json"
    }
    
    #this gets the response
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None
    
activities = get_Activities()
if activities:
    # Initialize a defaultdict to store yearly distances
    yearly_distances = defaultdict(float)
    
    for activity in activities:
        try:
            # Extract year from the activity date
            activity_date = datetime.fromisoformat(activity['start_date_local'].replace('Z', '+00:00'))
            year = activity_date.year
            
            # Add distance to yearly total (convert to kilometers if needed)
            if 'distance' in activity and activity['distance'] is not None:
                yearly_distances[year] += activity['distance'] / 1000
        except KeyError as e:
            print(f"Missing field in activity: {e}")
        except Exception as e:
            print(f"Error processing activity: {e}")
  
    
    # Print yearly totals
    print("Yearly Cycling Distances:")
    print("-----------------------")
    for year in sorted(yearly_distances.keys()):
        total_distance = yearly_distances[year]
        print(f"{year}: {total_distance:.2f} km")
    
    # Print overall total
    total_distance = sum(yearly_distances.values())
    print("-----------------------")
    print(f"Total Distance: {total_distance:.2f} km")
else:
    print("Failed to retrieve activities.")

/Users/avomref/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Yearly Cycling Distances:
-----------------------
2024: 2903.67 km
2025: 864.68 km
-----------------------
Total Distance: 3768.35 km
